# Failure Case Eval
This file is a helper tool to evaluate the predictions of the Mixmatch model.
The file reads in the predictions of the Mixmatch model, and display the images that the model predictly wrong. Make it easier to figure out the edge cases for the model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
import IPython.display as display

tf.enable_eager_execution()

In [ ]:
# Global constants
INPUT_PRED_DIR = '../Mixmatch/results-streetview_v4_64.1@250-200.csv'
INPUT_TF_DIR = '../Mixmatch/ML_DATA/streetview_v4_64-test.tfrecord'

In [ ]:
# Parse the label and image data from the tfrecord file
def parse_tf_records(file_dir):
    raw_image_dataset = tf.data.TFRecordDataset(file_dir)

    # Create a dictionary describing the features.
    image_feature_description = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image': tf.io.FixedLenFeature([], tf.string),
    }

    def _parse_image_function(example_proto):
      # Parse the input tf.Example proto using the dictionary above.
      return tf.io.parse_single_example(example_proto, image_feature_description)

    parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
    
    return parsed_image_dataset

In [ ]:
parsed_image_dataset = parse_tf_records(INPUT_TF_DIR)

In [ ]:
# Read in the predictions from the output of the resnet model. The first 2 columns are the labels, and the other 2 columns are the prediction probabilities.
results = pd.read_csv(INPUT_PRED_DIR, header=None)
labels = results.iloc[:,0:1].to_numpy()
pred = results.iloc[:,1:]

In [ ]:
# Enumerate through every example, and display the wrong prediction images.
for i, image_features in enumerate(parsed_image_dataset):
    label = str(int(labels[i, 0]))
    prediction = str(pred.iloc[i, :].idxmax() - 1)

    if label != prediction:
        print("True label: " + label + "\nPred label: " + prediction)
        image_raw = image_features['image'].numpy()
        display.display(display.Image(data=image_raw))